In [1]:
import pandas as pd
import pymc3 as pm
import numpy as np
import matplotlib.pyplot as plt

# Load Dataset

In [2]:
df = pd.read_csv('./heart-failure.csv', sep=',', index_col=False)
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270,0
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271,0
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278,0
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280,0


# Function

In [3]:
def df_train(number_register):
    df_train = df.copy()
    df_train = df_train.head(number_register)
    return df_train

def df_test(number_register):
    df_test = df.copy()
    df_test = df_test.tail(number_register)
    return df_test

def min_value(column_name):
    min_value = df[column_name].min()
    print(f"El valor min {column_name} es {min_value}")
    return min_value

def max_value(column_name):
    max_value = df[column_name].max()
    print(f"El valor max {column_name} es {max_value}")
    return max_value

def data_gen_min_value(column_name):
    max_value = sintetico[column_name].min()
    print(f"El valor min {column_name} es {max_value}")
    return max_value

def data_gen_max_value(column_name):
    max_value = sintetico[column_name].max()
    print(f"El valor max {column_name} es {max_value}")
    return max_value

# Clean Data

In [4]:
# Nulos en las columnas
df_null = df.isnull().sum()
df_null

age                         0
anaemia                     0
creatinine_phosphokinase    0
diabetes                    0
ejection_fraction           0
high_blood_pressure         0
platelets                   0
serum_creatinine            0
serum_sodium                0
sex                         0
smoking                     0
time                        0
DEATH_EVENT                 0
dtype: int64

In [5]:
# int 
df[['age','platelets']] = df[['age','platelets']].astype(int)
#df

In [6]:
# rename target variable
df = df.rename(columns={'DEATH_EVENT':'target'})

In [7]:
# verifiying int in df
df

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
294,62,0,61,1,38,1,155000,1.1,143,1,1,270,0
295,55,0,1820,0,38,0,270000,1.2,139,0,0,271,0
296,45,0,2060,1,60,0,742000,0.8,138,0,0,278,0
297,45,0,2413,0,38,0,140000,1.4,140,1,1,280,0


# Analize data real

### Numeric columns

In [8]:
min_value_age = min_value('age')
max_value_age = max_value('age')

El valor min age es 40
El valor max age es 95


In [9]:
min_value_creatine = min_value('creatinine_phosphokinase')
max_value_creatine = max_value('creatinine_phosphokinase')

El valor min creatinine_phosphokinase es 23
El valor max creatinine_phosphokinase es 7861


In [10]:
min_value_ejection = min_value('ejection_fraction')
max_value_ejection = max_value('ejection_fraction')

El valor min ejection_fraction es 14
El valor max ejection_fraction es 80


In [11]:
min_value_platelets = min_value('platelets')
max_value_platelets = max_value('platelets')

El valor min platelets es 25100
El valor max platelets es 850000


In [12]:
min_value_creatinine = min_value('serum_creatinine')
max_value_creatinine = max_value('serum_creatinine')

El valor min serum_creatinine es 0.5
El valor max serum_creatinine es 9.4


In [13]:
min_value_sodium = min_value('serum_sodium')
max_value_sodium = max_value('serum_sodium')

El valor min serum_sodium es 113
El valor max serum_sodium es 148


In [14]:
min_value_time = min_value('time')
max_value_time = max_value('time')

El valor min time es 4
El valor max time es 285


### Binary columns

In [15]:
min_value_anemia = min_value('anaemia')
max_value_anemia = max_value('anaemia')

El valor min anaemia es 0
El valor max anaemia es 1


In [16]:
min_value_diabetes = min_value('diabetes')
max_value_diabetes = max_value('diabetes')

El valor min diabetes es 0
El valor max diabetes es 1


In [17]:
min_value_pressure = min_value('high_blood_pressure')
max_value_pressure = max_value('high_blood_pressure')

El valor min high_blood_pressure es 0
El valor max high_blood_pressure es 1


In [18]:
min_value_sex = min_value('sex')
max_value_sex = max_value('sex')

El valor min sex es 0
El valor max sex es 1


In [19]:
min_value_smoking = min_value('smoking')
max_value_smoking = max_value('smoking')

El valor min smoking es 0
El valor max smoking es 1


# Split train - test data

In [20]:
df_train = df_train(275)
#print(df_train)
df_train

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,target
0,75,0,582,0,20,1,265000,1.9,130,1,0,4,1
1,55,0,7861,0,38,0,263358,1.1,136,1,0,6,1
2,65,0,146,0,20,0,162000,1.3,129,1,1,7,1
3,50,1,111,0,20,0,210000,1.9,137,1,0,7,1
4,65,1,160,1,20,0,327000,2.7,116,0,0,8,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
270,44,0,582,1,30,1,263358,1.6,130,1,1,244,0
271,51,0,582,1,40,0,221000,0.9,134,0,0,244,0
272,67,0,213,0,38,0,215000,1.2,133,0,0,245,0
273,42,0,64,0,40,0,189000,0.7,140,1,0,245,0


In [21]:
#df_test = df_test(25)
#df_test

# Creating Generative Bayesian model

In [22]:
# Split predictors variables(x) and target variable(y)

x_real = df_train.drop('target', axis=1)
y_real = df_train['target']

In [23]:
# verify matrix 
x_shape = x_real.shape
y_shape = y_real.shape

x_shape

(275, 12)

## Split data

In [24]:
#binary data

binary_data = x_real[['anaemia','diabetes','high_blood_pressure','sex','smoking']]

# array only values
binary_data_values = binary_data.values


In [25]:
#numeric data

numeric_data = x_real[['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time']]

# array only values
numeric_data_values = numeric_data.values
numeric_data_values

array([[7.500e+01, 5.820e+02, 2.000e+01, ..., 1.900e+00, 1.300e+02,
        4.000e+00],
       [5.500e+01, 7.861e+03, 3.800e+01, ..., 1.100e+00, 1.360e+02,
        6.000e+00],
       [6.500e+01, 1.460e+02, 2.000e+01, ..., 1.300e+00, 1.290e+02,
        7.000e+00],
       ...,
       [6.700e+01, 2.130e+02, 3.800e+01, ..., 1.200e+00, 1.330e+02,
        2.450e+02],
       [4.200e+01, 6.400e+01, 4.000e+01, ..., 7.000e-01, 1.400e+02,
        2.450e+02],
       [6.000e+01, 2.570e+02, 3.000e+01, ..., 1.000e+00, 1.370e+02,
        2.450e+02]])

In [26]:
# target variable
y_target = y_real
y_target_values = y_target.values

#### Normal model bayesian
with pm.Model() as generative_model:
    x = pm.Normal('x', mu=x_real.mean(), sd=1.0, shape=(275, 12))
    x_sintetico = pm.Normal('x_sintetico', mu=x, sd=1.0, shape=(275,12))
    
    y = pm.Normal('y', mu=y_real.mean(), sd=1.0, shape=(275))
    y_sintetico = pm.Normal('y_sintetico', mu=y, sd=1.0, shape=(275))

In [27]:
with pm.Model() as generative_model:
    x_binaria = pm.Bernoulli('x_binaria',p=binary_data, shape=binary_data.shape)
    x_numeric = pm.HalfNormal('x_numeric',sd=1, shape=numeric_data.shape)
    y_target = pm.Bernoulli('y_target',p=y_target, shape=y_target.shape)

In [28]:
# generating 
with generative_model:
    samplers = pm.sample(1000, tune=1000)

/home/stefy/anaconda3/envs/ml-env/lib/python3.7/site-packages/deprecat/classic.py:215: FutureWarning: In v4.0, pm.sample will return an `arviz.InferenceData` object instead of a `MultiTrace` by default. You can pass return_inferencedata=True or return_inferencedata=False to be safe and silence this warning.
  return wrapped_(*args_, **kwargs_)
Multiprocess sampling (4 chains in 4 jobs)
CompoundStep
>BinaryGibbsMetropolis: [x_binaria, y_target]
>NUTS: [x_numeric]


Sampling 4 chains for 1_000 tune and 1_000 draw iterations (4_000 + 4_000 draws total) took 153 seconds.
/home/stefy/anaconda3/envs/ml-env/lib/python3.7/site-packages/arviz/stats/diagnostics.py:586: RuntimeWarning: invalid value encountered in double_scalars
  (between_chain_variance / within_chain_variance + num_samples - 1) / (num_samples)


In [29]:
binary_generative = samplers['x_binaria']
binary_generative

array([[[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]],

       [[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]],

       [[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]],

       ...,

       [[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]],

       [[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]],

       [[0, 0, 1, 1, 0],
        [0, 0, 0, 1, 0],
        [0, 0, 0, 1, 1],
        ...,
        [0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0],
        [1, 1, 0, 1, 1]]

In [30]:
numeric_generative = samplers['x_numeric']
numeric_generative

array([[[0.33323969, 1.17926199, 1.84275717, ..., 1.27104964,
         0.9283213 , 0.44527599],
        [0.07643311, 0.36832541, 1.76630227, ..., 0.86576697,
         0.99280858, 0.0310868 ],
        [0.68932464, 0.40863364, 1.95763981, ..., 1.3960977 ,
         0.65088239, 1.32907693],
        ...,
        [0.83291362, 0.58367515, 0.24656819, ..., 1.098169  ,
         0.51379303, 1.76572572],
        [0.37878797, 0.29318332, 0.38889615, ..., 0.71380211,
         0.2387074 , 0.99827956],
        [0.36621973, 0.424954  , 0.78735671, ..., 2.1656481 ,
         0.63852722, 0.91653746]],

       [[0.7989619 , 0.2690579 , 0.30242055, ..., 0.66345784,
         0.42350785, 0.5853376 ],
        [0.45333211, 1.69920629, 0.23916199, ..., 0.90967814,
         1.00557919, 2.09806734],
        [0.73955589, 1.05446339, 0.23578883, ..., 0.55284361,
         0.39604855, 0.40562904],
        ...,
        [0.26067443, 0.8738572 , 1.97281064, ..., 0.69823268,
         1.47166092, 0.47019189],
        [0.0

In [31]:
target_generative = samplers['y_target']
target_generative

array([[1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       ...,
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0],
       [1, 1, 1, ..., 0, 0, 0]])

In [32]:
#x_sintetico_generado

In [33]:
#x_sintetico_generado = x_sintetico_generado.reshape(-1, x_sintetico_generado.shape[-1])
#sintetico = pd.DataFrame(x_sintetico_generado, columns=['age','anaemia','creatinine_phosphokinase','diabetes','ejection_fraction','high_blood_pressure','platelets','serum_creatinine','serum_sodium','sex','smoking','time'])
#sintetico

In [34]:
#binary_generative

In [35]:
binary_generative = binary_generative.reshape(-1, binary_generative.shape[-1])
binary_generative = pd.DataFrame(binary_generative, columns=['anaemia','diabetes','high_blood_pressure','sex','smoking'])
type(binary_generative)

binary_generative

,anaemia,diabetes,high_blood_pressure,sex,smoking
0,0,0,1,1,0
1,0,0,0,1,0
2,0,0,0,1,1
3,1,0,0,1,0
4,1,1,0,0,0
...,...,...,...,...,...
1099995,0,1,1,1,1
1099996,0,1,0,0,0
1099997,0,0,0,0,0
1099998,0,0,0,1,0


In [36]:
len(binary_generative)

1100000

In [37]:
binary_generative_null = binary_generative.isnull().sum()
binary_generative_null

anaemia                0
diabetes               0
high_blood_pressure    0
sex                    0
smoking                0
dtype: int64

In [58]:
numeric_generative = numeric_generative.reshape(-1, numeric_generative.shape[-1])
#numeric_generative_number = numeric_generative * 100
numeric_generative = pd.DataFrame(numeric_generative*100, columns=['age','creatinine_phosphokinase','ejection_fraction','platelets','serum_creatinine','serum_sodium','time'])
type(numeric_generative)

numeric_generative

AttributeError: 'DataFrame' object has no attribute 'reshape'

In [39]:
len(numeric_generative)

1100000

In [40]:
numeric_generative_null = numeric_generative.isnull().sum()
numeric_generative_null

age                         0
creatinine_phosphokinase    0
ejection_fraction           0
platelets                   0
serum_creatinine            0
serum_sodium                0
time                        0
dtype: int64

In [41]:
target_generative = target_generative.reshape(-1, 1)
target_generative = pd.DataFrame(target_generative, columns=['target'])
type(target_generative)

target_generative

,target
0,1
1,1
2,1
3,1
4,1
...,...
1099995,0
1099996,0
1099997,0
1099998,0


In [42]:
len(target_generative)

1100000

In [43]:
target_generative_null = target_generative.isnull().sum()
target_generative_null

target    0
dtype: int64

In [44]:
sintetico = pd.concat([numeric_generative, binary_generative, target_generative], axis=1)
sintetico

,age,creatinine_phosphokinase,ejection_fraction,platelets,serum_creatinine,serum_sodium,time,anaemia,diabetes,high_blood_pressure,sex,smoking,target
0,33.323969,117.926199,184.275717,50.770784,127.104964,92.832130,44.527599,0,0,1,1,0,1
1,7.643311,36.832541,176.630227,76.306377,86.576697,99.280858,3.108680,0,0,0,1,0,1
2,68.932464,40.863364,195.763981,1.351585,139.609770,65.088239,132.907693,0,0,0,1,1,1
3,21.011068,50.552002,22.327027,68.157343,6.345500,66.186549,44.004542,1,0,0,1,0,1
4,17.593333,59.961834,19.666639,106.879445,160.876408,17.767123,38.025853,1,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1099995,11.904353,9.698147,115.413952,29.153250,60.831008,12.940873,75.053718,0,1,1,1,1,0
1099996,65.854118,31.599621,23.869032,73.967743,164.710022,72.292275,124.686526,0,1,0,0,0,0
1099997,161.908179,165.016370,67.595003,103.530378,67.594395,114.501046,194.812138,0,0,0,0,0,0
1099998,104.160651,46.916699,38.295351,118.019507,30.114421,32.507932,48.693058,0,0,0,1,0,0


# Analyze generic data 

## predictor variables

### Numeric data

In [45]:
data_gen_min_value_age = data_gen_min_value('age')
data_gen_max_value_age = data_gen_max_value('age')

El valor min age es 0.00019949276584228226
El valor max age es 516.8628675979857


In [46]:
data_gen_min_value_creatine = data_gen_min_value('creatinine_phosphokinase')
data_gen_max_value_creatine = data_gen_max_value('creatinine_phosphokinase')

El valor min creatinine_phosphokinase es 2.2275268213934233e-05
El valor max creatinine_phosphokinase es 496.3940850393938


In [47]:
data_gen_min_value_ejection = data_gen_min_value('ejection_fraction')
data_gen_max_value_ejection = data_gen_max_value('ejection_fraction')

El valor min ejection_fraction es 0.0003718619025349293
El valor max ejection_fraction es 502.4042133775105


In [48]:
data_gen_min_value_platelets = data_gen_min_value('platelets')
data_gen_max_value_platelets = data_gen_max_value('platelets')

El valor min platelets es 6.383024638745924e-05
El valor max platelets es 495.3757164700109


In [49]:
data_gen_min_value_creatine = data_gen_min_value('serum_creatinine')
data_gen_max_value_creatine = data_gen_max_value('serum_creatinine')

El valor min serum_creatinine es 0.00028364329648861435
El valor max serum_creatinine es 487.73979039317084


In [50]:
data_gen_min_value_sodium = data_gen_min_value('serum_sodium')
data_gen_max_value_sodium = data_gen_max_value('serum_sodium')

El valor min serum_sodium es 6.54021250078657e-05
El valor max serum_sodium es 558.1276273794383


In [51]:
data_gen_min_value_time = data_gen_min_value('time')
data_gen_max_value_time = data_gen_max_value('time')

El valor min time es 5.4622708488764834e-05
El valor max time es 484.43772397669136


### Binary data

In [52]:
data_gen_min_value_anaemia = data_gen_min_value('anaemia')
data_gen_max_value_anaemia = data_gen_max_value('anaemia')

El valor min anaemia es 0
El valor max anaemia es 1


In [53]:
data_gen_min_value_diabetes = data_gen_min_value('diabetes')
data_gen_max_value_diabetes = data_gen_max_value('diabetes')

El valor min diabetes es 0
El valor max diabetes es 1


In [54]:
data_gen_min_value_pressure = data_gen_min_value('high_blood_pressure')
data_gen_max_value_pressure = data_gen_max_value('high_blood_pressure')

El valor min high_blood_pressure es 0
El valor max high_blood_pressure es 1


In [55]:
data_gen_min_value_sex = data_gen_min_value('sex')
data_gen_max_value_sex = data_gen_max_value('sex')

El valor min sex es 0
El valor max sex es 1


In [56]:
data_gen_min_value_smoking = data_gen_min_value('smoking')
data_gen_max_value_smoking = data_gen_max_value('smoking')

El valor min smoking es 0
El valor max smoking es 1


## Target variable

In [57]:
y_sintetico_generado

NameError: name 'y_sintetico_generado' is not defined

In [ ]:
#y_sintetico_generado = y_sintetico_generado.reshape(-1, y_sintetico_generado.shape[-1])
y_sintetico_generado = y_sintetico_generado.flatten()
sintetic = pd.DataFrame({'target': y_sintetico_generado})
sintetic

# Otro modelo generativo

In [ ]:
# generating 
with generative_model:
    samplers = pm.sample(1000, tune=1000)